# Tutorial data

In [1]:
# import pandas as pd

# data_path = "data/train/day_0_0.001_train.csv"

# df = pd.read_csv(data_path, header=None)
# df.head()

In [2]:
# example = df.loc[1, 0]
# example

In [3]:
# features = example.split('\t')
# len(features)
# features

# Read s3 orc data

In [1]:
column_names = []
with open(f'./schema/column_name_mobivista.txt', 'r') as f:
    for line in f:
        column_names.append(line.split(' ')[1].strip())
print(column_names)

['_11001', '_11002', '_11003', '_11004', '_11007', '_11008', '_11021', '_11022', '_11023', '_11024', '_11041', '_11042', '_11043', '_11044', '_11045', '_11046', '_11061', '_11062', '_11063', '_11064', '_11065', '_11066', '_11081', '_11082', '_11083', '_11084', '_11085', '_11086', '_11601', '_11602', '_11603', '_12001', '_12002', '_12003', '_12004', '_12005', '_12006', '_20001', '_20002', '_20003', '_20101', '_20102', '_20201', '_20202', '_20203', '_20204', '_20205', '_20206', '_20207', '_20208', '_20209', '_20210', '_30001', '_30002', '_30003', '_30004', '_30005', '_30006', '_30201', '_30202', '_30203', '_30204', '_30205', '_30206', '_30207', '_40001', '_40002', '_40003', '_40004', '_40005', '_40201', '_40202', '_40203', '_40204', '_40205', '_40206', '_40207', '_40208', '_40209', '_40210', '_40211', '_40212', '_40213', '_40214', '_40215', '_40231', '_40301', '_40302', '_40303', '_40304', '_40305', '_40306', '_40307', '_40321', '_40322', '_40323', '_40324', '_50801', '_50802', '_50805',

In [2]:
import metaspore as ms

# train_dataset_path = ROOT_DIR + '/data/train/day_0_0.001_train.csv'

file_base_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/'
num_files = 10
file_names = [f'part-{str(i).zfill(5)}-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc' for i in range(num_files)]
train_dataset_path = [file_base_path + file_name for file_name in file_names]

# train_dataset_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/part-00000-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc'
# train_dataset_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/'

spark_confs = {
    'spark.eventLog.enabled':'true',
    'spark.executor.memory': '10g',
    'spark.driver.memory': '10g',
    "spark.driver.maxResultSize": "10g",
    "spark.sql.files.ignoreCorruptFiles": "true"
}

spark_session = ms.spark.get_session(local=True,
                                     batch_size=100,
                                     worker_count=2,
                                     server_count=2,
                                     log_level='WARN',
                                     spark_confs=spark_confs)

train_dataset = ms.input.read_s3_csv(spark_session, 
                                     train_dataset_path, 
                                     format='orc',
                                     shuffle=False,
                                     delimiter='\t', 
                                     multivalue_delimiter="\001", 
                                     column_names=column_names,
                                     multivalue_column_names=column_names[:-1])

# train_dataset = spark_session.read.orc(train_dataset_path)

# train_dataset.printSchema()

/opt/conda/envs/metaspore_py39/lib/python3.9/site-packages/pyarrow/compute.py:196: RuntimeWarning: Python binding for StringBKDRHashFunctionOption not exposed
  warnings.warn("Python binding for {} not exposed"
/opt/conda/envs/metaspore_py39/lib/python3.9/site-packages/pyarrow/compute.py:196: RuntimeWarning: Python binding for BKDRHashCombineFunctionOption not exposed
  warnings.warn("Python binding for {} not exposed"
24/07/19 02:56:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


ignore shuffle


In [3]:
import pandas as pd

df = train_dataset.toPandas()
df.head()

24/07/19 02:56:43 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/07/19 02:58:31 ERROR TaskSetManager: Total size of serialized results of 24 tasks (10.2 GiB) is bigger than spark.driver.maxResultSize (10.0 GiB)
24/07/19 02:58:31 WARN TaskSetManager: Lost task 23.0 in stage 1.0 (TID 173) (ip-172-31-0-41.us-west-2.compute.internal executor driver): TaskKilled (Tasks result size has exceeded maxResultSize)
/opt/conda/envs/metaspore_py39/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:137: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true, but has reached the error below and can not continue. Note that 'spark.sql.execution.arrow.pyspark.fallback.enabled' does not have an effect on failures in the middle of computation.
  An error occurred while calling o23828.getResult.
: org.apache.spark.Spark

Py4JJavaError: An error occurred while calling o23828.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:97)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:93)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands

24/07/19 02:58:31 WARN TaskSetManager: Lost task 24.0 in stage 1.0 (TID 174) (ip-172-31-0-41.us-west-2.compute.internal executor driver): TaskKilled (Stage cancelled)
24/07/19 02:58:32 WARN TaskSetManager: Lost task 26.0 in stage 1.0 (TID 176) (ip-172-31-0-41.us-west-2.compute.internal executor driver): TaskKilled (Stage cancelled)


In [7]:
df.shape

(124491, 324)

In [12]:
num_positive = df[df['label']=='1'].shape[0]
num_negative = df[df['label']=='0'].shape[0]
print(f"num_positive: {num_positive}, num_negative: {num_negative}, ratio: {num_positive/num_negative*100}")

num_positive: 2919, num_negative: 121572, ratio: 2.4010462935544368


In [15]:
columns = df.columns.tolist()
print(columns)


['_11001', '_11002', '_11003', '_11004', '_11007', '_11008', '_11021', '_11022', '_11023', '_11024', '_11041', '_11042', '_11043', '_11044', '_11045', '_11046', '_11061', '_11062', '_11063', '_11064', '_11065', '_11066', '_11081', '_11082', '_11083', '_11084', '_11085', '_11086', '_11601', '_11602', '_11603', '_12001', '_12002', '_12003', '_12004', '_12005', '_12006', '_20001', '_20002', '_20003', '_20101', '_20102', '_20201', '_20202', '_20203', '_20204', '_20205', '_20206', '_20207', '_20208', '_20209', '_20210', '_30001', '_30002', '_30003', '_30004', '_30005', '_30006', '_30201', '_30202', '_30203', '_30204', '_30205', '_30206', '_30207', '_40001', '_40002', '_40003', '_40004', '_40005', '_40201', '_40202', '_40203', '_40204', '_40205', '_40206', '_40207', '_40208', '_40209', '_40210', '_40211', '_40212', '_40213', '_40214', '_40215', '_40231', '_40301', '_40302', '_40303', '_40304', '_40305', '_40306', '_40307', '_40321', '_40322', '_40323', '_40324', '_50801', '_50802', '_50805',

In [20]:
total_hash = 0
feature_hash = {}
feature_value = {}

for i, row in df.iterrows():
    for column in columns[:-1]:
        if column in feature_value:
            feature_value[column].append(row[column].shape[0])
        else:
            feature_value[column] = [row[column].shape[0]]
        
        for item in row[column]:
            if '\003' in item:
                hash, weight = item.split('\003')
            else:
                hash, weight = '', 0
            if column in feature_hash:
                feature_hash[column].add(hash)
            else:
                feature_hash[column] = set(hash)

df_feature = pd.DataFrame(feature_value)
df_feature.describe()



ValueError: not enough values to unpack (expected 2, got 1)

In [18]:
len(df.loc[0, '_11001'])

3